In [ ]:
import MyPackage.MyGui as MyGui
import paramiko
import textfsm
import ipaddress
import logging
import sys
import time
from multiprocessing.pool import ThreadPool
import multiprocessing
from tkinter import Tk
import ctypes
import pandas
import openpyxl
import socket
import os
import datetime
import shutil

jump_server = "10.251.131.6"
_USERNAME = ""
_PASSWORD = ""
TIMEOUT = 15
LOCAL_IP_ADDRESS = "127.0.0.1"

In [ ]:
def ip_check(ip) -> bool:
    """
    Takes in an IP Address as a string.
    Checks that the IP address is a valid one.
    Returns True or false.
    :param ip: Example: 192.168.1.1
    :return: Boolean
    """
    try:
        ipaddress.ip_address(ip)
        return True
    except ValueError:
        print(f"ip_check function ValueError: "
                  f"IP Address: {ip} is an invalid address. Please check and try again!")
        return False

def jump_session(ip, username=_USERNAME, password=_PASSWORD) -> "SSH Session + Jump Session + Connection Status":
    """
    Takes in an IP Address as a string.
    Connects to the IP address through a jump host using SSH.
    Returns the SSH session, The jump Session and
    a boolean value that represents the state of the connection.
    :param username:
    :param password:
    :param ip: The IP Address you wish to connect to.
    :return: SSH Session + Jump Session + Connection Status(Boolean).
    """
    if not ip_check(ip):
        print(f"open_session function error: "
                  f"ip Address {ip} is not a valid Address. Please check and restart the script!", )
        return None, None, False
    try:
        print(f"Jump Session Function: Trying to establish a connection to: {ip}")
        jump_box = paramiko.SSHClient()
        jump_box.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        jump_box.connect(jump_server, username=_USERNAME, password=_PASSWORD)
        jump_box_transport = jump_box.get_transport()
        src_address = (LOCAL_IP_ADDRESS, 22)
        destination_address = (ip, 22)
        jump_box_channel = jump_box_transport.open_channel("direct-tcpip", destination_address, src_address,
                                                           timeout=TIMEOUT, )
        target = paramiko.SSHClient()
        target.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        target.connect(hostname=ip, username=username, password=password,
                       sock=jump_box_channel, timeout=TIMEOUT, auth_timeout=TIMEOUT, banner_timeout=TIMEOUT)
        print(f"Jump Session Function: Connection to IP: {ip} established")
        return target, jump_box, True
    except paramiko.ssh_exception.AuthenticationException:
        if ANSWER_REDO == "Yes":
            if ip not in AUTHENTICATION_ERRORS:
                AUTHENTICATION_ERRORS.append(ip)
                print(f"Jump Session Function Error: Authentication to IP: {ip} failed! "
                              f"Retrying using 'answer' credentials.")
                jump_session(ip, A_USERNAME, A_PASSWORD)
            elif ip in AUTHENTICATION_ERRORS:
                print(f"Jump Session Function Error: Authentication to IP: {ip} failed! "
                              f"Please check your ip, username and password.")
                return None, None, False
        else:
            AUTHENTICATION_ERRORS.append(ip)
            print(f"Jump Session Function Error: Authentication to IP: {ip} failed! "
                      f"Please check your ip, username and password.")
            return None, None, False
    except paramiko.ssh_exception.NoValidConnectionsError:
        CONNECTION_ERRORS.append(ip)
        print(f"Jump Session Function Error: Unable to connect to IP: {ip}!")
        return None, None, False
    except (ConnectionError, TimeoutError):
        CONNECTION_ERRORS.append(ip)
        print(f"Jump Session Function Error: Connection or Timeout error occurred for IP: {ip}!")
        return None, None, False
    except Exception as err:
        CONNECTION_ERRORS.append(ip)
        print(f"Jump Session Function Error: An unknown error occurred for IP: {ip}!\n{err}")
        return None, None, False